# Flat_Finder - aplikacja do wyszukiwania mieszkań w odniesieniu do punktów

Projektem jest stworzenie aplikacji wykorzystującej scrapowanie w języku Python, celem której będzie pomoc w znalezieniu mieszkań w określonych miastach. Aplikacja będzie integrować się z zewnętrznym API w celu zlokalizowania istotnych punktów, takich jak miejsca pracy użytkownika, a następnie zaprezentuje mieszkania znajdujące się w ich bliskim sąsiedztwie. Mieszkania i ich odlełość będą określane przy pomocy zewnętrznego API TomTom oraz obliczania odległości przy pomocy krzywej ziemskiej na podstawie koordynatów w celu zooptymalizowania działania aplikacji

Import bibliotek:
- pandas - operacje na ramkach danych. Wczytanie oraz przygotowywanie danych do późniejszego wykorzystania a bazie danych
- re - praca z wyrażeniami regularnymi. Wykorzystywane do przetworzenia danych z tytułów oraz opisów ogłoszeń w celu zlokalizowania mieszkań

In [2]:
import pandas as pd
import re

Import oraz podgląd ramki danych z mieszkaniami

In [3]:
df = pd.read_csv("mieszkanka5960.csv")
df['opis'].head()

0    REZERWACJA! do sprzedania mieszkanie w Świdnic...
1    do sprzedania mieszkanie znajdujące się w cent...
2    Na sprzedaż   lokal  o powierzchni 49m2 + 9m2 ...
3    Do sprzedania dwupoziomowe mieszkanie w Świdni...
4    Przedstawiam Państwu słoneczne mieszkanie przy...
Name: opis, dtype: object

Wykorzystanie biblioteki *regex* w celu wyciągnięcia informacji z ogłoszeń zawierające informacje gdzie znajduje się dane mieszkanie

In [4]:
import re

def extract_street_names(text):
    # Wzorzec do wyciągania nazw ulic oraz potencjalnie numerów budynków
    street_pattern = re.compile(
        r'(?<!\bplac\s)(ul\.|ulica|ulicy|al\.|aleja|alei|osiedle|osiedla|ulice|Placu|Plac|osiedlem|os.|osiedlu)\s+(\d*\s*)?([A-ZĄĆĘŁŃÓŚŹŻ][a-ząćęłńóśźż]+(?:\s+[A-ZĄĆĘŁŃÓŚŹŻ][a-ząćęłńóśźż]+)*)\s*(\d*)', re.IGNORECASE)
    
    # Sprawdzenie, czy tekst jest typu string
    if isinstance(text, str):
        # Wyszukiwanie nazw ulic w tekście
        matches = street_pattern.findall(text)
        street_names = []
        for match in matches:
            street_type, _, street_name, street_number = match
            full_street_name = f"{street_type} {street_name}"
            if street_number:
                full_street_name += f" {street_number}"
            street_names.append(full_street_name.strip())
        return street_names
    else:
        return []

# Tworzenie kolumny 'street_names' na podstawie kolumn 'title' i 'opis'
df['street_names'] = df.apply(lambda row: extract_street_names(row['title']) + extract_street_names(row['opis']), axis=1)

Podgląd wyciągniętych z tytułów oraz opisów adresów

In [5]:
print(df['street_names'][0:30])

0                                                    []
1                                                    []
2                                                    []
3                                        [Os. Zawiszów]
4                                      [ul. Kosynierów]
5                      [ul. Kosynierów, ul. Kosynierów]
6                      [ul. Kosynierów, ul. Kosynierów]
7                                         [ul. Centrum]
8                          [ul. Franciszkańska, osi zł]
9     [ul. Pionierów, ulicy Pionierów, osi zaledwie ...
10                                        [os. Młodych]
11                                 [ul. Wodnej, osi zł]
12                                                   []
13                                                   []
14                                                   []
15                                                   []
16                                        [Os. Młodych]
17                                              

In [6]:
'''import re

def extract_street_names(text):
    # Wzorzec do wyciągania nazw ulic oraz potencjalnie numerów budynków
    street_pattern = re.compile(
        r'(?<!\bplac\s)(ul\.|ulica|ulicy|al\.|aleja|alei|osiedle|osiedlu|osiedla|osiedlem|placu|)\s+(\d*\s*)?([A-ZĄĆĘŁŃÓŚŹŻ][a-ząćęłńóśźż]+(?:\s+[A-ZĄĆĘŁŃÓŚŹŻ][a-ząćęłńóśźż]+)*)\s*(\d*)', re.IGNORECASE)
    
    # Sprawdzenie, czy tekst jest typu string
    if isinstance(text, str):
        # Wyszukiwanie nazw ulic w tekście
        matches = street_pattern.findall(text)
        street_names = []
        for match in matches:
            street_type, _, street_name, street_number = match
            full_street_name = f"{street_type} {street_name}"
            if street_number:
                full_street_name += f" {street_number}"
            street_names.append(full_street_name.strip())
        return street_names
    else:
        return []

# Przykładowy tekst
text = "Mieszkanie na czwartym piętrze w bloku mieszkalnym na osiedlu Piastowskim."

# Wywołanie funkcji
result = extract_street_names(text)
print(result)
df['street_names'][0:30]

'''

'import re\n\ndef extract_street_names(text):\n    # Wzorzec do wyciągania nazw ulic oraz potencjalnie numerów budynków\n    street_pattern = re.compile(\n        r\'(?<!\x08plac\\s)(ul\\.|ulica|ulicy|al\\.|aleja|alei|osiedle|osiedlu|osiedla|osiedlem|placu|)\\s+(\\d*\\s*)?([A-ZĄĆĘŁŃÓŚŹŻ][a-ząćęłńóśźż]+(?:\\s+[A-ZĄĆĘŁŃÓŚŹŻ][a-ząćęłńóśźż]+)*)\\s*(\\d*)\', re.IGNORECASE)\n    \n    # Sprawdzenie, czy tekst jest typu string\n    if isinstance(text, str):\n        # Wyszukiwanie nazw ulic w tekście\n        matches = street_pattern.findall(text)\n        street_names = []\n        for match in matches:\n            street_type, _, street_name, street_number = match\n            full_street_name = f"{street_type} {street_name}"\n            if street_number:\n                full_street_name += f" {street_number}"\n            street_names.append(full_street_name.strip())\n        return street_names\n    else:\n        return []\n\n# Przykładowy tekst\ntext = "Mieszkanie na czwartym piętrze

Usunięcie pustych wierszy odpowiadających mieszkaniom dla których nie udało się określić lokalizacji

In [7]:
# Usunięcie wierszy zawierających puste ciągi znaków w kolumnie 'street_names'
df = df[df['street_names'].str.len() > 1]

In [8]:
df['street_names']

5                        [ul. Kosynierów, ul. Kosynierów]
6                        [ul. Kosynierów, ul. Kosynierów]
8                            [ul. Franciszkańska, osi zł]
9       [ul. Pionierów, ulicy Pionierów, osi zaledwie ...
11                                   [ul. Wodnej, osi zł]
                              ...                        
5955    [ul. Akademicka, Osiedle AkademickieNajważniej...
5956    [ul. Akademicka, Osiedle AkademickieNajważniej...
5957    [ul. Akademicka, Osiedle AkademickieNajważniej...
5958    [ul. Akademicka, Osiedle AkademickieNajważniej...
5959    [ul. Akademicka, Osiedle AkademickieNajważniej...
Name: street_names, Length: 3787, dtype: object

Utworzenie maski dla wierszy, w których każdy element zawiera co najmniej jedną wielką literę

In [9]:
mask = df['street_names'].apply(lambda x: any(char.isupper() for char in ', '.join(x)))
df = df[mask]
print(df)

        id      region                                             title  \
5        5   Świdnica,          Nowe mieszkanie Świdnica, ul. Kosynierów   
6        6   Świdnica,          Nowe mieszkanie Świdnica, ul. Kosynierów   
8        8   Świdnica,           Mieszkanie Świdnica, ul. Franciszkańska   
9        9   Świdnica,                               Mieszkanie Świdnica   
11      11   Świdnica,                               Mieszkanie Świdnica   
...    ...         ...                                               ...   
5955  5955  Bydgoszcz,  Nowe mieszkanie Bydgoszcz Fordon, ul. Akademicka   
5956  5956  Bydgoszcz,  Nowe mieszkanie Bydgoszcz Fordon, ul. Akademicka   
5957  5957  Bydgoszcz,  Nowe mieszkanie Bydgoszcz Fordon, ul. Akademicka   
5958  5958  Bydgoszcz,  Nowe mieszkanie Bydgoszcz Fordon, ul. Akademicka   
5959  5959  Bydgoszcz,  Nowe mieszkanie Bydgoszcz Fordon, ul. Akademicka   

                      total_price additional_price_per_m2  \
5     363 146\n      \n   

In [10]:
df['street_names']

5                        [ul. Kosynierów, ul. Kosynierów]
6                        [ul. Kosynierów, ul. Kosynierów]
8                            [ul. Franciszkańska, osi zł]
9       [ul. Pionierów, ulicy Pionierów, osi zaledwie ...
11                                   [ul. Wodnej, osi zł]
                              ...                        
5955    [ul. Akademicka, Osiedle AkademickieNajważniej...
5956    [ul. Akademicka, Osiedle AkademickieNajważniej...
5957    [ul. Akademicka, Osiedle AkademickieNajważniej...
5958    [ul. Akademicka, Osiedle AkademickieNajważniej...
5959    [ul. Akademicka, Osiedle AkademickieNajważniej...
Name: street_names, Length: 3738, dtype: object

In [11]:
import pandas as pd

# Łączenie elementów listy w jedną linię
df['street_names'] = df['street_names'].apply(lambda x: ', '.join(x))

# Usunięcie wszystkiego po przecinku w kolumnie
df['street_names'] = df['street_names'].apply(lambda x: x.split(',')[0])

In [12]:
df

,id,region,title,total_price,additional_price_per_m2,opis,Lokalizacja,Powierzchnia w m2,Liczba pokoi,Piętro,...,Usytuowanie względem stron świata,prysznic,komórka lokatorska,siła,strych,gimnazjum,tramwaj,lotnisko,metro,street_names
5,5,"Świdnica,","Nowe mieszkanie Świdnica, ul. Kosynierów",363 146\n \n zł,"7 050,01 zł/m2",Przedstawiam Państwu słoneczne mieszkanie przy...,"Świdnica, ...","51,51 m2",3,2,...,"wschodnie, zachodnie, południowe",prysznic,komórka lokatorska,siła,strych,NaN,NaN,NaN,NaN,ul. Kosynierów
6,6,"Świdnica,","Nowe mieszkanie Świdnica, ul. Kosynierów",278 207\n \n zł,"7 150,01 zł/m2",Przedstawiam Państwu mieszkanie przy ul. Kosyn...,"Świdnica, ...","38,91 m2",2,1,...,"wschodnie, zachodnie, południowe",prysznic,komórka lokatorska,siła,strych,NaN,NaN,NaN,NaN,ul. Kosynierów
8,8,"Świdnica,","Mieszkanie Świdnica, ul. Franciszkańska",290 000\n \n zł,"3 493,55 zł/m2",Prezentuję Państwu na sprzedaż mieszkanie do r...,"Świdnica, ...","83,01 m2",3,3,...,"wschodnie, zachodnie, południowe",prysznic,komórka lokatorska,siła,strych,NaN,NaN,NaN,NaN,ul. Franciszkańska
9,9,"Świdnica,",Mieszkanie Świdnica,299 000\n \n zł,"8 305,56 zł/m2","Mieszkanie 2 pokoje, ul. Pionierów, Świdnica.O...","Świdnica, ...",36 m2,2,1,...,"wschodnie, zachodnie, południowe",prysznic,komórka lokatorska,siła,strych,NaN,NaN,NaN,NaN,ul. Pionierów
11,11,"Świdnica,",Mieszkanie Świdnica,360 000\n \n zł,"7 708,78 zł/m2",REZERWACJA!!!!!!!!Zapraszam do zapoznania się ...,"Świdnica, ...","46,70 m2",2,1,...,"wschodnie, zachodnie, południowe",prysznic,komórka lokatorska,siła,strych,NaN,NaN,NaN,NaN,ul. Wodnej
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5955,5955,"Bydgoszcz,","Nowe mieszkanie Bydgoszcz Fordon, ul. Akademicka",Zapytaj o cenę,NaN,O inwestycji Murapol Osiedle AkademickieNajważ...,"Bydgoszcz, ...","58,18 m2",4,4,...,"południowe, północne",prysznic,komórka lokatorska,siła,strych,gimnazjum,tramwaj,lotnisko,metro,ul. Akademicka
5956,5956,"Bydgoszcz,","Nowe mieszkanie Bydgoszcz Fordon, ul. Akademicka",Zapytaj o cenę,NaN,O inwestycji Murapol Osiedle AkademickieNajważ...,"Bydgoszcz, ...","43,41 m2",2,4,...,"południowe, północne",prysznic,komórka lokatorska,siła,strych,gimnazjum,tramwaj,lotnisko,metro,ul. Akademicka
5957,5957,"Bydgoszcz,","Nowe mieszkanie Bydgoszcz Fordon, ul. Akademicka",Zapytaj o cenę,NaN,O inwestycji Murapol Osiedle AkademickieNajważ...,"Bydgoszcz, ...","40,72 m2",2,4,...,"południowe, północne",prysznic,komórka lokatorska,siła,strych,gimnazjum,tramwaj,lotnisko,metro,ul. Akademicka
5958,5958,"Bydgoszcz,","Nowe mieszkanie Bydgoszcz Fordon, ul. Akademicka",Zapytaj o cenę,NaN,O inwestycji Murapol Osiedle AkademickieNajważ...,"Bydgoszcz, ...","43,86 m2",2,4,...,"południowe, północne",prysznic,komórka lokatorska,siła,strych,gimnazjum,tramwaj,lotnisko,metro,ul. Akademicka


In [13]:
print(df['street_names'])

5           ul. Kosynierów
6           ul. Kosynierów
8       ul. Franciszkańska
9            ul. Pionierów
11              ul. Wodnej
               ...        
5955        ul. Akademicka
5956        ul. Akademicka
5957        ul. Akademicka
5958        ul. Akademicka
5959        ul. Akademicka
Name: street_names, Length: 3738, dtype: object


Kolejne wykorzystanie wydarzenia regularnego w celu dostosowania znalezionych adresów do znalezienia ich współrzędnych geograficznych przy pomocy api *OpenStreetMap Nominatim*

In [14]:
df['street_names'] = df['street_names'].str.replace(r'\b(?:ul(?:ica|ic|icy|ice)?\.?)\b', '', regex=True)

df['street_names'] = df['street_names'].str.replace(r'\b(?:al(?:eja|\.|e|i|ej|om|ą|y)?)\b', 'aleja', regex=True)

df['cstreet_names'] = df['street_names'].str.replace(r'\b(?:Pl(?:ac|\.|.|ace|acu|aców|acowi|acom|acami|acach|acem)?)\b', 'plac', regex=True)

df['street_names'] = df['street_names'].str.replace(r'\b(?:Pomnik|pomniki|pomnika|pomników|pomnikowi|pomnikom|pomniku|pomnikiem|pomnikami|pomnikach)\\b', 'pomnik', regex=True)

df['cstreet_names'] = df['street_names'].str.replace(r'\b(?:Osiedle|osiedla|osiedlu|osiedlem|osiedlach|osiedlami|osiedlom|osiedlowi|osiedlowi)\\b', 'osiedle', regex=True)

In [15]:
df['street_names']

5           . Kosynierów
6           . Kosynierów
8       . Franciszkańska
9            . Pionierów
11              . Wodnej
              ...       
5955        . Akademicka
5956        . Akademicka
5957        . Akademicka
5958        . Akademicka
5959        . Akademicka
Name: street_names, Length: 3738, dtype: object

Zapis ramki danych w formie kopii zapasowej

In [16]:
df.to_csv('streests.csv')

Usunięcie niepotrzebnych znaków pozwalających na utworzenie kolumn o wartościach liczbowych

In [17]:
df['region'] = df['region'].str.replace(',', '')
df['total_price'] = df['total_price'].str.replace(r'\s+|\\n', '', regex=True)
df['total_price'] = df['total_price'].str.replace('zł', '')
df['additional_price_per_m2'] = df['additional_price_per_m2'].str.replace('zł/m2', '')
df['additional_price_per_m2'] = df['additional_price_per_m2'].str.replace(',', '.')
df['Powierzchnia w m2'] = df['Powierzchnia w m2'].str.replace(',', '.')
df['Powierzchnia w m2'] = df['Powierzchnia w m2'].str.replace(' m2', '')
df

,id,region,title,total_price,additional_price_per_m2,opis,Lokalizacja,Powierzchnia w m2,Liczba pokoi,Piętro,...,prysznic,komórka lokatorska,siła,strych,gimnazjum,tramwaj,lotnisko,metro,street_names,cstreet_names
5,5,Świdnica,"Nowe mieszkanie Świdnica, ul. Kosynierów",363146,7 050.01,Przedstawiam Państwu słoneczne mieszkanie przy...,"Świdnica, ...",51.51,3,2,...,prysznic,komórka lokatorska,siła,strych,NaN,NaN,NaN,NaN,. Kosynierów,. Kosynierów
6,6,Świdnica,"Nowe mieszkanie Świdnica, ul. Kosynierów",278207,7 150.01,Przedstawiam Państwu mieszkanie przy ul. Kosyn...,"Świdnica, ...",38.91,2,1,...,prysznic,komórka lokatorska,siła,strych,NaN,NaN,NaN,NaN,. Kosynierów,. Kosynierów
8,8,Świdnica,"Mieszkanie Świdnica, ul. Franciszkańska",290000,3 493.55,Prezentuję Państwu na sprzedaż mieszkanie do r...,"Świdnica, ...",83.01,3,3,...,prysznic,komórka lokatorska,siła,strych,NaN,NaN,NaN,NaN,. Franciszkańska,. Franciszkańska
9,9,Świdnica,Mieszkanie Świdnica,299000,8 305.56,"Mieszkanie 2 pokoje, ul. Pionierów, Świdnica.O...","Świdnica, ...",36,2,1,...,prysznic,komórka lokatorska,siła,strych,NaN,NaN,NaN,NaN,. Pionierów,. Pionierów
11,11,Świdnica,Mieszkanie Świdnica,360000,7 708.78,REZERWACJA!!!!!!!!Zapraszam do zapoznania się ...,"Świdnica, ...",46.70,2,1,...,prysznic,komórka lokatorska,siła,strych,NaN,NaN,NaN,NaN,. Wodnej,. Wodnej
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5955,5955,Bydgoszcz,"Nowe mieszkanie Bydgoszcz Fordon, ul. Akademicka",Zapytajocenę,NaN,O inwestycji Murapol Osiedle AkademickieNajważ...,"Bydgoszcz, ...",58.18,4,4,...,prysznic,komórka lokatorska,siła,strych,gimnazjum,tramwaj,lotnisko,metro,. Akademicka,. Akademicka
5956,5956,Bydgoszcz,"Nowe mieszkanie Bydgoszcz Fordon, ul. Akademicka",Zapytajocenę,NaN,O inwestycji Murapol Osiedle AkademickieNajważ...,"Bydgoszcz, ...",43.41,2,4,...,prysznic,komórka lokatorska,siła,strych,gimnazjum,tramwaj,lotnisko,metro,. Akademicka,. Akademicka
5957,5957,Bydgoszcz,"Nowe mieszkanie Bydgoszcz Fordon, ul. Akademicka",Zapytajocenę,NaN,O inwestycji Murapol Osiedle AkademickieNajważ...,"Bydgoszcz, ...",40.72,2,4,...,prysznic,komórka lokatorska,siła,strych,gimnazjum,tramwaj,lotnisko,metro,. Akademicka,. Akademicka
5958,5958,Bydgoszcz,"Nowe mieszkanie Bydgoszcz Fordon, ul. Akademicka",Zapytajocenę,NaN,O inwestycji Murapol Osiedle AkademickieNajważ...,"Bydgoszcz, ...",43.86,2,4,...,prysznic,komórka lokatorska,siła,strych,gimnazjum,tramwaj,lotnisko,metro,. Akademicka,. Akademicka


Utworzenie 3 zmiennych ze zmiennej lokalizacja

In [18]:
df[['Miasto', 'Powiat', 'Wojewodztwo']] = df['Lokalizacja'].str.split(", ", expand=True)

print(df['Wojewodztwo'])

5                                               dolnoś...
6                                               dolnoś...
8                                               dolnoś...
9                                               dolnoś...
11                                              dolnoś...
                              ...                        
5955                                            kujaws...
5956                                            kujaws...
5957                                            kujaws...
5958                                            kujaws...
5959                                            kujaws...
Name: Wojewodztwo, Length: 3738, dtype: object


Usunięcie białych znaków z kolumny Województw

In [19]:
df['Wojewodztwo'] = df['Wojewodztwo'].str.replace(r'\s+', '', regex=True)
df['Wojewodztwo']

5             dolnośląskie
6             dolnośląskie
8             dolnośląskie
9             dolnośląskie
11            dolnośląskie
               ...        
5955    kujawsko-pomorskie
5956    kujawsko-pomorskie
5957    kujawsko-pomorskie
5958    kujawsko-pomorskie
5959    kujawsko-pomorskie
Name: Wojewodztwo, Length: 3738, dtype: object

Stworznie kolumny umożliwiającej wprowadzanie zapytyań do API nominatim za pomocą prostego iterowania po kolejnych jej elementach

In [20]:
df['Polaczona_Kolumna'] = df['region']   +' '+  df['Wojewodztwo'] +', '+  df['street_names'] 
list = df['Polaczona_Kolumna']
list

5              Świdnica dolnośląskie, . Kosynierów
6              Świdnica dolnośląskie, . Kosynierów
8          Świdnica dolnośląskie, . Franciszkańska
9               Świdnica dolnośląskie, . Pionierów
11                 Świdnica dolnośląskie, . Wodnej
                           ...                    
5955    Bydgoszcz kujawsko-pomorskie, . Akademicka
5956    Bydgoszcz kujawsko-pomorskie, . Akademicka
5957    Bydgoszcz kujawsko-pomorskie, . Akademicka
5958    Bydgoszcz kujawsko-pomorskie, . Akademicka
5959    Bydgoszcz kujawsko-pomorskie, . Akademicka
Name: Polaczona_Kolumna, Length: 3738, dtype: object

Geokodowanie adresów dla wycięgniętych adresów dostosowanych do api *Nominatim*
Dla każdego adresu podejmowana jest próba geokodowania. Jeśli lokalizacja zostanie znaleziona, jej współrzędne są wyświetlane i dodawane do listy wynikowej. Jeśli nie, do listy dodawany jest wpis "brak". Na końcu wyświetlane są wyniki oraz liczba pomyślnych operacji i błędów. Dodatkowo, wprowadzone jest opóźnienie między żądaniami, aby uniknąć zbyt częstego wysyłania żądań.

In [21]:
import time
from geopy.geocoders import Nominatim

# Lista do przechowywania wyników geokodowania
geocoded_addresses = []

# Inicjalizacja Nominatim
geolocator = Nominatim(user_agent = "abcd")

# Liczniki iterazji
success_count = 0
error_count = 0

for address in list:
    try:
        location = geolocator.geocode(address)
        
        if location:
            print(f"Address: {address}, Latitude: {location.latitude}, Longitude: {location.longitude}")
            geocoded_addresses.append((location.latitude, location.longitude))
            success_count += 1
            print(success_count)
        else:
            print(f"No geocoding result for {address}")
            geocoded_addresses.append(("brak"))
            error_count += 1
    except Exception as e:
        print(f"An error occurred: {e}")
        geocoded_addresses.append(("brak"))
        error_count += 1
        print(error_count)
    
    time.sleep(3)

# Wyświetlamy zgeokodowane adresy i liczniki
for item in geocoded_addresses:
    print(item)

print(f"\nLiczba pomyślnie zgeokodowanych adresów: {success_count}")
print(f"Liczba adresów, które nie zostały zgeokodowane lub wystąpił błąd: {error_count}")

Address: Świdnica dolnośląskie, . Kosynierów, Latitude: 50.8498854, Longitude: 16.4964833
1
Address: Świdnica dolnośląskie, . Kosynierów, Latitude: 50.8498854, Longitude: 16.4964833
2


KeyboardInterrupt: 

Wybór kolumn które finalnie trafią do bazy danych

In [ ]:
dftest = df[['id', 'region', 'title', 'total_price',
             'additional_price_per_m2', 'opis', 'Lokalizacja', 
             'Powierzchnia w m2', 'Liczba pokoi', 'Piętro', 
             'Forma własności', 'Rok budowy', 'Typ zabudowy', 
             'Liczba pięter w budynku', 'Forma kuchni']]

Usunięcie lokacji których adresów nie udało się otrzymać przy pomocy geokodowania

In [ ]:
dftest['latlon'] = geocoded_addresses
dftest =  dftest[dftest['latlon'] != 'brak']
dftest['latlon']

Zmiana formatu w którym zapisane są współrzędne geograficzne

In [ ]:
dftest['latlon'] = dftest['latlon'].astype(str).str.replace('(', '')
dftest['latlon'] = dftest['latlon'].astype(str).str.replace(')', '')
dftest['latlon'] = dftest['latlon'].astype(str).str.replace(' ', '')

Zapis do ramki danych

In [ ]:
dftest.to_csv('latlon.csv')